<a href="https://colab.research.google.com/github/ijeoma91/Batch_Processing_Pipeline/blob/main/Batch_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=3a606e82fd1621e877780c36f88438fcb529836f6ec012b26c5fa93e3c2d65dd
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58749 sha256=7cf288e5ccd334856fec385aef2e6587b12232aba4bbeed8b7faf4f9bd0a626c
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark


In [3]:
pip install cassandra-driver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 17.4 MB/s eta 0:00:00


In [4]:
pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 6.6 MB/s eta 0:00:00


In [5]:
pip install timedelta

  Preparing metadata (setup.py) ... done
  Created wheel for timedelta: filename=timedelta-2020.12.3-py3-none-any.whl size=1556 sha256=6ebfde4731f5b6983ec3e2ac8a2d5713d49cd257afe0a51b1fb4f5446aacf462
  Stored in directory: /root/.cache/pip/wheels/be/c0/1f/843bd73679dc7dfb26758c4effc6e273d3eebf2da2a35082b1
Successfully built timedelta


In [6]:
pip install jupyter-dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.0 MB/s eta 0:00:00


In [7]:
pip install pyngrok

In [8]:
! ngrok authtoken 2eWmBka8I8O6dJGkz6BO9XhP2hN_c7jsE4CwP8fjbqwidHdE
#from pyngrok import ngrok

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
import plotly.express as px
from jupyter_dash import JupyterDash
#import dash_core_components as dcc
from dash import dcc
from dash import html
import dash_html_components as html

In [11]:

import os
packages = ["pandas", "schedule", "cassandra-driver"]
for packages in packages:
  os.system(f'pip install {packages}')

In [12]:
import pandas as pd
import datetime
#from datetime import datatime, timedelta
import time
import timedelta

In [13]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [23]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder.appName("My Data Batch Processing").getOrCreate()

# Replace 'dataset path' with the actual path to your CSV file
Batch_df = spark.read.csv("/content/iot_telemetry_data.csv", header=True, sep=',')

# Show the original DataFrame
print("Original DataFrame:")
Batch_df.show()

# Function to clean and uppercase column names
def clean_and_uppercase(name):
    # Remove unwanted characters (example: spaces and special characters)
    cleaned_name = ''.join(e for e in name if e.isalnum())
    # Make sure the first letter is uppercase
    cleaned_name = cleaned_name[0].upper() + cleaned_name[1:]
    return cleaned_name

# Rename columns
for col_name in Batch_df.columns:
    new_col_name = clean_and_uppercase(col_name)
    Batch_df = Batch_df.withColumnRenamed(col_name, new_col_name)

# Show the modified DataFrame
print("Modified DataFrame:")
Batch_df.show()

Original DataFrame:
+--------------------+--------------------+--------------------+---------+--------------------+----------+--------------------+--------------------+
|      "ts,""device""|              ""co""|        ""humidity""|""light""|             ""lpg""|""motion""|           ""smoke""|           ""temp"""|
+--------------------+--------------------+--------------------+---------+--------------------+----------+--------------------+--------------------+
|"1.59451209438597...|""0.0049559386483...|            ""51.0""|""false""|""0.0076508222705...| ""false""|""0.0204112701224...|           ""22.7"""|
|"1.59451209473556...|""0.0028400886071...|            ""76.0""|""false""|""0.0051143834009...| ""false""|""0.0132748367048...|""19.700000762939...|
|"1.59451209807357...|""0.0049760123404...|            ""50.9""|""false""|""0.0076732274063...| ""false""|""0.0204751255761...|           ""22.6"""|
|"1.59451209958914...|""0.0044030268296...|""76.800003051757...| ""true""|""0.00702333

In [24]:
# let's find how many device that has the highest temperature thereby grouping
from pyspark.sql.functions import *
from pyspark.sql.types import *


Batch_df.groupBy("Tsdevice").agg(countDistinct("Temp").alias("devices_count")).show()

+--------------------+-------------+
|            Tsdevice|devices_count|
+--------------------+-------------+
|"1.59452496005098...|            1|
|"1.59455160255505...|            1|
|"1.59482533195824...|            1|
|"1.59483897429002...|            1|
|"1.59487127495136...|            1|
|"1.59487128679406...|            1|
|"1.59453095631002...|            1|
|"1.59457600258627...|            1|
|"1.59457845300001...|            1|
|"1.59459310240455...|            1|
|"1.59465516904282...|            1|
|"1.59466583917450...|            1|
|"1.59483756707918...|            1|
|"1.59459084007783...|            1|
|"1.59455132218290...|            1|
|"1.59465414743181...|            1|
|"1.59478061339207...|            1|
|"1.59481121126203...|            1|
|"1.59483560211041...|            1|
|"1.59460478521667...|            1|
+--------------------+-------------+
only showing top 20 rows



In [25]:
from pyspark.sql import functions as F

# Assuming batch_df is your DataFrame and "Humidity" is the column you want to cast to float
Batch_df = Batch_df.withColumn("Humidity", F.col("Humidity").cast("float"))

# Show the result
Batch_df.show()

+--------------------+--------------------+--------+---------+--------------------+---------+--------------------+--------------------+
|            Tsdevice|                  Co|Humidity|    Light|                 Lpg|   Motion|               Smoke|                Temp|
+--------------------+--------------------+--------+---------+--------------------+---------+--------------------+--------------------+
|"1.59451209438597...|""0.0049559386483...|    NULL|""false""|""0.0076508222705...|""false""|""0.0204112701224...|           ""22.7"""|
|"1.59451209473556...|""0.0028400886071...|    NULL|""false""|""0.0051143834009...|""false""|""0.0132748367048...|""19.700000762939...|
|"1.59451209807357...|""0.0049760123404...|    NULL|""false""|""0.0076732274063...|""false""|""0.0204751255761...|           ""22.6"""|
|"1.59451209958914...|""0.0044030268296...|    NULL| ""true""|""0.0070233371458...|""false""|""0.0186282253770...|           ""27.0"""|
|"1.59451210176123...|""0.0049673636419...|    N

In [27]:
#initial_data(batch_df)

spark.stop()

In [26]:
from pyspark.sql import SparkSession, functions as F
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

def initialize_cassandra_session():
    """
    Initializes and returns a Cassandra session.
    Ensure the Cassandra cluster is accessible through the specified host and port.
    """
    auth_provider = PlainTextAuthProvider(username='username', password='cassandra')
    cluster = Cluster(['0.tcp.eu.ngrok.io'], port=18718, auth_provider=auth_provider)
    session = cluster.connect()
    return session

def process_and_insert_data(dat, Batch_df):
    """
    Processes the provided DataFrame and inserts the aggregated data into Cassandra.
    """
    # Create aggregates
    agg_df = Batch_df.groupBy("device").agg(
        F.avg('co').alias('avg_co'),
        F.avg('humidity').alias('avg_humidity'),
        F.avg('lpg').alias('avg_lpg'),
        F.avg('smoke').alias('avg_smoke'),
        F.sum(F.when(F.col('light') == 'true', 1).otherwise(0)).alias('light_true'),
        F.sum(F.when(F.col('light') == 'false', 1).otherwise(0)).alias('light_false'),
        F.sum(F.when(F.col('motion') == 'true', 1).otherwise(0)).alias('motion_true'),
        F.sum(F.when(F.col('motion') == 'false', 1).otherwise(0)).alias('motion_false')
    ).collect()

    session = initialize_cassandra_session()
    session.execute("CREATE KEYSPACE IF NOT EXISTS batch WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 3}")
    session.execute("""CREATE TABLE IF NOT EXISTS batch.sensors (
        device_id text PRIMARY KEY, date timestamp, lpg float, smoke float, humidity float, co float,
        light_true int, light_false int, motion_true int, motion_false int)""")

    # Insert data
    for row in agg_df:
        session.execute(
            """
            INSERT INTO batch.sensors (device_id, date, lpg, smoke, humidity, co, light_true, light_false, motion_true, motion_false)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """,
            (row['device'], dat, row['avg_lpg'], row['avg_smoke'], row['avg_humidity'], row['avg_co'],
             row['light_true'], row['light_false'], row['motion_true'], row['motion_false'])
        )

    print("Data inserted into Cassandra for date:", dat)

def main(spark, Batch_df):
    # Example usage with a specific date
    process_and_insert_data('2024-04-02', Batch_df)  # Replace '2023-01-01' with your actual date variable



if Batch_df== "_main_":
    spark = SparkSession.builder.appName("My_data_Batch_Processing").getOrCreate()
    # Assuming you've loaded your DataFrame 'df' here
    # df = spark.read.csv("your_csv_file_path.csv", header=True, inferSchema=True)


    main(spark, Batch_df)  # Pass 'df' as an argument to 'main'